In [1]:
import pickle
import json
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set(style='ticks', font_scale = 1.5)
sns.set(rc={'figure.figsize': (18, 6)})
sns.set_style('whitegrid')

plt.rcParams.update({'font.size': 12})

models = ["dbscan", "hdbscan", "birch", "kmeans-minibatch"]
norm_models = ["dbscan", "hdbscan"]
suite = 'nlp'
num_datasets = 9
df_ref = pickle.load(open('nlp_dfs.pkl', 'rb'))


In [2]:
from fanova import fANOVA

import ConfigSpace
from ConfigSpace.hyperparameters import CategoricalHyperparameter, UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, Constant
from hydra import compose, initialize
from omegaconf import OmegaConf

from clustering_hyperparameters.utils.type_utils import get_type_from_str
metadata_cols = ["adjusted_rand_score", 
                "trial_status",
                "generator_model",
                "trial_index",
                "arm_name",
                "compute_time"]

res_df = { m : pd.DataFrame() for m in models } 
for model in models:
    all_df_model = pd.concat([df_ref['random'][model], df_ref['bayesian'][model]])
    all_df_model_groups = all_df_model.groupby('dataset')
    

    with initialize(config_path="../src/clustering_hyperparameters/conf", job_name="app"):
        for dataset_index in range(num_datasets):
            cfg = compose(config_name="config", overrides=["model="+model, "suite="+suite, "dataset_index="+str(dataset_index)])
            cfg.dataset_index = str(cfg.dataset_index)
            cfg = OmegaConf.to_container(cfg, resolve=True)
            dataset = cfg["suite"]["datasets"][dataset_index]["name"]
            dataset_with_norm = dataset 
            if model not in norm_models:
                dataset_with_norm += "_normalized"
            try :
                dataset_df = all_df_model_groups.get_group(dataset_with_norm)
            except KeyError as e:
                continue
            print("model:", model, "dataset_index:", dataset_index, "dataset", dataset)
            
            
            params = [ x for x in cfg["model"]["params"] if x['type']=='range'] 
            param_names = [ x['name'] for x in params ]
            cs = ConfigSpace.ConfigurationSpace()
            for ind, param in enumerate(params):
                log_scale = param['log_scale'] if 'log_scale' in param else False
                if param['value_type'] == "int":
                    cs.add_hyperparameter(UniformIntegerHyperparameter(param["name"], lower=param['bounds'][0], upper=param['bounds'][1], log=log_scale))
                elif param['value_type'] == "float":
                    cs.add_hyperparameter(UniformFloatHyperparameter(param["name"], lower=param['bounds'][0], upper=param['bounds'][1], log=log_scale))
                elif param['value_type'] == "str":
                    cs.add_hyperparameter(CategoricalHyperparameter(param["name"], choices=param['choices']))
                    
            X , Y = dataset_df[param_names], dataset_df['adjusted_rand_score'].to_numpy()
            f = fANOVA(X = X, Y = Y, config_space=cs, seed=0)
            res_dict = {'dataset': dataset}
            for param in param_names: 
                try:
                    res_dict[param] = f.quantify_importance((param,))[(param,)]['individual importance']
                except Exception:
                    res_dict[param] = 0.0
            res_df[model] = res_df[model].append(res_dict, ignore_index=True)  


/opt/homebrew/anaconda3/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


model: dbscan dataset_index: 0 dataset AGNews-paraphrase-mpnet
model: dbscan dataset_index: 1 dataset AGNews-stsb-distilroberta
model: dbscan dataset_index: 2 dataset AGNews-glove
model: dbscan dataset_index: 3 dataset DBpedia-paraphrase-mpnet
model: dbscan dataset_index: 4 dataset DBpedia-stsb-distilroberta
model: dbscan dataset_index: 5 dataset DBpedia-glove
model: dbscan dataset_index: 6 dataset YahooAnswers-paraphrase-mpnet
model: dbscan dataset_index: 7 dataset YahooAnswers-stsb-distilroberta
model: dbscan dataset_index: 8 dataset YahooAnswers-glove
model: hdbscan dataset_index: 0 dataset AGNews-paraphrase-mpnet
model: hdbscan dataset_index: 1 dataset AGNews-stsb-distilroberta
model: hdbscan dataset_index: 2 dataset AGNews-glove
model: hdbscan dataset_index: 3 dataset DBpedia-paraphrase-mpnet
model: hdbscan dataset_index: 4 dataset DBpedia-stsb-distilroberta
model: hdbscan dataset_index: 5 dataset DBpedia-glove
model: hdbscan dataset_index: 6 dataset YahooAnswers-paraphrase-mpnet


In [3]:
from IPython.display import display
all_dfs = []
for model in models:
    df = res_df[model].copy()
    df['encoder'] = df['dataset'].apply(lambda x: "-".join(x.split("-")[1:]))
    df['dataset'] = df['dataset'].apply(lambda x: x.split("-")[0])
    df.set_index(['dataset', 'encoder'], inplace=True)
    df = df.T.copy()
    df['model'] = [model] * len(df)
    df.index.name = 'hyperparameters'
    df.reset_index(level=0, inplace=True)
    df.set_index(['model', 'hyperparameters'], inplace=True)
    all_dfs.append(df)

all_dfs = pd.concat(all_dfs)
all_dfs = all_dfs * 100
display(all_dfs)
with open('importances_nlp.tex', 'w') as f:
    f.write(all_dfs.to_latex(float_format="%.3f"))

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


dataset                                              AGNews  \
encoder                                    paraphrase-mpnet   
model            hyperparameters                              
dbscan           eps                              37.568645   
                 min_samples                       6.477639   
hdbscan          cluster_selection_epsilon         0.262562   
                 min_cluster_size                  4.927874   
                 min_samples                      79.091124   
birch            branching_factor                  5.720532   
                 threshold                        76.903115   
kmeans-minibatch batch_size                        2.225524   
                 init_size                         4.647261   
                 max_iter                          4.326036   
                 max_no_improvement                1.532757   
                 n_init                           21.323396   
                 reassignment_ratio                2.265045   
                 tol                               1.501356   

dataset                                                                   \
encoder                                    stsb-distilroberta      glove   
model            hyperparameters                                           
dbscan           eps                                37.324431  50.991090   
                 min_samples                         8.604143  11.112048   
hdbscan          cluster_selection_epsilon           1.577297   9.125589   
                 min_cluster_size                   12.548380  10.096574   
                 min_samples                        51.025720  52.466761   
birch            branching_factor                   12.066211   5.877264   
                 threshold                          53.877544  65.074782   
kmeans-minibatch batch_size                          4.174537   0.787536   
                 init_size                           8.632586   3.930700   
                 max_iter                            1.283839   8.644913   
                 max_no_improvement                  1.832627   3.137869   
                 n_init                             10.128481  26.487425   
                 reassignment_ratio                  5.280752   2.294919   
                 tol                                 5.122947   2.732547   

dataset                                             DBpedia  \
encoder                                    paraphrase-mpnet   
model            hyperparameters                              
dbscan           eps                              58.151817   
                 min_samples                      10.376788   
hdbscan          cluster_selection_epsilon         2.520172   
                 min_cluster_size                 18.541017   
                 min_samples                      32.549169   
birch            branching_factor                 14.575826   
                 threshold                        43.376474   
kmeans-minibatch batch_size                        1.831552   
                 init_size                        17.680704   
                 max_iter                          3.200539   
                 max_no_improvement                1.534678   
                 n_init                           11.390918   
                 reassignment_ratio                3.135838   
                 tol                               6.287917   

dataset                                                                   \
encoder                                    stsb-distilroberta      glove   
model            hyperparameters                                           
dbscan           eps                                78.663948  65.720691   
                 min_samples                         2.716656   9.357979   
hdbscan          cluster_selection_epsilon          50.794100  67.427803   
                 min_cluster_size                    2.263034   2.135467   
                 min_samples   